In [1]:
import urllib
urllib.request.urlretrieve("http://www.parlgov.org/static/data/development-utf-8/view_election.csv","elections.csv")

('elections.csv', <http.client.HTTPMessage at 0x7f3538bc8a20>)

In [2]:
import pandas as pd
import numpy as np

In [3]:
elections = pd.read_csv('elections.csv')

In [4]:
elections.head()

,country_name_short,country_name,election_type,election_date,vote_share,seats,seats_total,party_name_short,party_name,party_name_english,left_right,country_id,election_id,previous_parliament_election_id,previous_cabinet_id,party_id
0,AUS,Australia,parliament,1901-03-30,44.4,32.0,75,PP,Protectionist Party,Protectionist Party,7.4000,33,731,NaN,NaN,1898
1,AUS,Australia,parliament,1901-03-30,34.2,26.0,75,FTP,Free Trade Party,Free Trade Party,6.0000,33,731,NaN,NaN,1938
2,AUS,Australia,parliament,1901-03-30,19.4,15.0,75,ALP,Australian Labor Party,Australian Labor Party,3.8833,33,731,NaN,NaN,1253
3,AUS,Australia,parliament,1901-03-30,1.4,1.0,75,none,no party affiliation,no party affiliation,NaN,33,731,NaN,NaN,1396
4,AUS,Australia,parliament,1901-03-30,0.6,1.0,75,one-seat,one seat,one seat,NaN,33,731,NaN,NaN,2299


List of countries:

In [5]:
print(np.unique(elections['country_name']))

['Australia' 'Austria' 'Belgium' 'Bulgaria' 'Canada' 'Croatia' 'Cyprus'
 'Czech Republic' 'Denmark' 'Estonia' 'Finland' 'France' 'Germany'
 'Greece' 'Hungary' 'Iceland' 'Ireland' 'Israel' 'Italy' 'Japan' 'Latvia'
 'Lithuania' 'Luxembourg' 'Malta' 'Netherlands' 'New Zealand' 'Norway'
 'Poland' 'Portugal' 'Romania' 'Slovakia' 'Slovenia' 'Spain' 'Sweden'
 'Switzerland' 'Turkey' 'United Kingdom']


Extract data for the Netherlands:

In [6]:
Country = "France"

In [7]:
electionsCountry = elections.loc[elections['country_name'] == Country]

In [8]:
dates = np.unique(electionsCountry['election_date'])

Convert string to datetime:

In [9]:
from datetime import datetime
plotDate=[]

for date in dates:
    plotDate.append(datetime.strptime(date, '%Y-%m-%d'))

Extract All seats, median of the seats and the LR score for the biggest party

In [10]:
medLine = []
AllSeats = []
AllLR = []
AllMaxLR =[]

for date in dates:
    E= np.array([])
    Ect =electionsCountry.loc[electionsCountry['election_date'] == date]
    seats = Ect['seats']
    left_right = Ect['left_right']
    
    maxlr = np.mean(Ect.loc[Ect['seats'] ==max(seats)]['left_right'])
    
    AllMaxLR.append(float(maxlr))
    
    AllSeats.append(seats.values)
    AllLR.append(left_right.values)
    for index,row in Ect.iterrows():
        if row['seats']>0 and not np.isnan(row['left_right']):
            E = np.append(E, np.full(int(row['seats']),row['left_right']))
    medLine.append(np.median(E))

Plotting!

In [11]:
import bokeh
from bokeh.plotting import figure, show, output_notebook

output_notebook()

p = figure(plot_width=800, plot_height=400,
           y_range=(0,10),x_axis_type='datetime')

p.step(plotDate, medLine, line_width=2, mode="after",color='black',legend ="Median Left-Right")
p.step(plotDate, AllMaxLR, line_width=2, mode="after",color='red',line_dash=[5,5],legend = "Biggest party")
for i, (seats, LR) in enumerate(zip(AllSeats,AllLR)):
    seatsN = 100*seats/np.sum(seats)
    p.scatter(plotDate[i],LR,size=seatsN,fill_alpha=0.5,legend='Parties')


p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Left-right score'

p.title.text = 'Elections for ' + Country
p.legend.location = "top_left"
p.legend.orientation = 'horizontal'
p.legend.click_policy = 'hide'
p.legend.inactive_fill_alpha = 0.6
show(p)

Loading BokehJS ...

## Plot for all countries

In [12]:
def PlotforCountry(Country):
    electionsCountry = elections.loc[elections['country_name'] == Country]
    
    #Get election dates
    dates = np.unique(electionsCountry['election_date'])
    plotDate=[]

    for date in dates:
        plotDate.append(datetime.strptime(date, '%Y-%m-%d'))
        
    #initialize    
    medLine = []
    AllSeats = []
    AllLR = []
    AllMaxLR =[]

    for date in dates:
        E= np.array([])
        Ect =electionsCountry.loc[electionsCountry['election_date'] == date]
        seats = Ect['seats']
        left_right = Ect['left_right']

        maxlr = np.mean(Ect.loc[Ect['seats'] ==max(seats)]['left_right'])

        AllMaxLR.append(float(maxlr))

        AllSeats.append(seats.values)
        AllLR.append(left_right.values)
        for index,row in Ect.iterrows():
            if row['seats']>0 and not np.isnan(row['left_right']):
                E = np.append(E, np.full(int(row['seats']),row['left_right']))
        medLine.append(np.median(E))
    
    p = figure(plot_width=800, plot_height=400,
           y_range=(0,10),x_axis_type='datetime')

    p.step(plotDate, medLine, line_width=2, mode="after",color='black',legend ="Median Left-Right")
    p.step(plotDate, AllMaxLR, line_width=2, mode="after",color='red',line_dash=[5,5],legend = "Biggest party")
    for i, (seats, LR) in enumerate(zip(AllSeats,AllLR)):
        seatsN = 100*seats/np.sum(seats)
        p.scatter(plotDate[i],LR,size=seatsN,fill_alpha=0.5,legend='Parties')


    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Left-right score'

    p.title.text = 'Elections for ' + Country
    p.legend.location = "top_left"
    p.legend.orientation = 'horizontal'
    p.legend.click_policy = 'hide'
    p.legend.inactive_fill_alpha = 0.6
    show(p)

In [13]:
for country in np.unique(elections['country_name']):
    PlotforCountry(country)